In [73]:
import io
import sys
import subprocess 
import copy

import numpy as np
import pyvista as pv

# make sure the user has at least Python 3.7, which we need for some capture_output
assert sys.version_info.major == 3, "This requires at least Python 3.7"
assert sys.version_info.minor >= 7, "This requires at least Python 3.7"# viz stuff
# Let's define some colors
RED = '#FF0000'
YELLOW = '#CCCC00'
GREEN = '#00DD00'
CYAN = '#00FFFF'
BLUE = '#0000FF'
MAGENTA = '#CC00CC'

LT_RED = '#FF3333'
DK_RED = '#990000'
ORANGE = '#FF8400'
LT_GREEN = '#44FF44'
DK_GREEN = '#009900'
NEON_GREEN = '#00FE94'
LT_BLUE = '#0088FF'
NEON_BLUE = '#7DF9FF'
LIGHT_COLOR = 'white'
BACKGROUND_COLOR = 'white'
GREY = '#AAAAAA'
LT_GREY = '#CCCCCC'

L_color = RED
W_color = DK_RED
C_color = LT_GREEN
V1_color = DK_GREEN
V2_color = LT_BLUE
# these next are for the lex display if desired
# vertex_size = 70
# W_color = RED
# C_color = GREEN
# V1_color = GREEN


def visualize_graph(layout, q, vertex_list, L, W, C, V1, V2, 
                    edge_sets, edge_opacities, edge_colors, line_widths, 
                    fname, camera_pos, camera_rot , show_labels, caption):
    print(caption)
    if q<17:
        edge_opacities[0] = 0.08
    elif q<27:
        edge_opacities[0] = 0.04
    elif q<47:
        edge_opacities[0] = 0.01
    elif q<63:
        edge_opacities[0] = 0.005
    elif q<129:
        edge_opacities[0] = 0.0005
    else:
        edge_opacities[0] = 0.0001
    vertex_size = 25
    vertex_size_smaller = vertex_size

    # camera_pos views: xy, xz, yz, yx, zx, zy, iso, can also define a camera_pos = (x,y,z) tuple
#     camera_pos = 'zx'
#     camera_rot = [0,5,15]
#     # overhead view
#     camera_pos = 'xy'
#     camera_rot = [0,0,180]
    pl = pv.Plotter(lighting=None, window_size=(2000, 2000))
    light1 = pv.Light(position=(10, 10., -5.0),
                       focal_point=(0, 0, 0),
                       color=LIGHT_COLOR,  # Color temp. 5400 K
                       intensity=1.5)
    light2 = pv.Light(position=(-10, 10.0, -5.0),
                       focal_point=(0, 0, 0),
                       color=LIGHT_COLOR,  # Color temp. 2850 K
                       intensity=0.75)
#     pl.add_light(light1)  # Lighting effect to be cast onto the object
#     pl.add_light(light2)  # Second Lighting effect to be cast onto the object
    hlight = pv.Light(light_type='headlight')
    pl.add_light(hlight)  # Second Lighting effect to be cast onto the object
    pl.set_background(BACKGROUND_COLOR)  # Set the background 
            
    # pdata is the basic vertex data positions
    vpos = []
    for i in range(len(vertex_list)):
        if layout == 'lex':
            vpos.append(vertex_list[i].pos[0]) 
        elif layout == 'good':
            vpos.append(vertex_list[i].pos[1]) 
        elif layout == 'cake':
            vpos.append(vertex_list[i].pos[2]) 
    pdata = pv.PolyData(vpos)     
# this adds point labels of vector values -- but placed on top of the points
    if (show_labels):
        point_labels = []
        for i in range(len(vertex_list)):
            point_labels.append(vertex_list[i].value)        
        pl.add_point_labels(pdata, point_labels, italic=False, bold=True, font_size=30,
                            point_color='black', point_size=0, text_color='black', 
                            shape=None, 
                            render_points_as_spheres=True,
                            always_visible=True, shadow=False)  
# this adds the other edge sets    
    other_pdata = []
    for i in range(len(edge_sets)):
        other_pdata.append(pv.PolyData(vpos))
        other_pdata[i].lines = edge_sets[i]
        pl.add_mesh(other_pdata[i],
                    color = edge_colors[i],
                    opacity = edge_opacities[i],
                    point_size=0,
                    line_width = line_widths[i],
                    render_points_as_spheres=True
        )        
    L_vertex_position=[]
    for i in L:
        L_vertex_position.append(vpos[L[0]])
    L_cloud = pv.PolyData(L_vertex_position)
    pl.add_mesh(L_cloud, 
                color=L_color, 
                point_size=vertex_size,
                render_points_as_spheres=True
               )
    W_vertex_position=[]
    for i in W:
        W_vertex_position.append(vpos[i])
    W_cloud = pv.PolyData(W_vertex_position)
    pl.add_mesh(W_cloud, 
                color=W_color, 
                point_size=vertex_size,
                render_points_as_spheres=True
               )
    C_vertex_position=[]
    for i in C:
        C_vertex_position.append(vpos[i])
    C_cloud = pv.PolyData(C_vertex_position)
    pl.add_mesh(C_cloud, 
                color=C_color, 
                point_size=vertex_size,
                render_points_as_spheres=True
               )
    V1_vertex_position=[]
    for i in V1:
        V1_vertex_position.append(vpos[i])
    V1_cloud = pv.PolyData(V1_vertex_position)
    pl.add_mesh(V1_cloud, 
                color=V1_color, 
                point_size=vertex_size_smaller,
                render_points_as_spheres=True
               )
    V2_vertex_position=[]
    for i in V2:
        V2_vertex_position.append(vpos[i])
    V2_cloud = pv.PolyData(V2_vertex_position)
    pl.add_mesh(V2_cloud, 
                color=V2_color, 
                point_size=vertex_size_smaller,
                render_points_as_spheres=True
               )    
    # Views: xy, xz, yz, yx, zx, zy, iso 
    pl.camera_position = camera_pos
    pl.camera.roll = camera_rot[0]
    pl.camera.azimuth = camera_rot[1]
    pl.camera.elevation = camera_rot[2]
    pl.camera.zoom(1)   
    # Save the plot as a screenshot.  We can change the output resolution if desired
    pl.screenshot(fname, window_size=[2000,2000])
    #pl.add_point_labels(vpos_sets[0], range(len(vpos_sets[0])), font_size=100)
    #pl.show_axes()
    pl.show()
